<a href="https://colab.research.google.com/github/rajvgokidi/Body-Measurement-Segmentation/blob/main/Detectron2_Keypoint_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core

#cloning github repos for AIMIRR resources received and detectron2 libraries
!git clone 'https://github.com/rajvgokidi/AIMIRR-Resources.git'
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")

#installing required packages based on requirement file
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

#unzipping Data.zip file from AIMIRR resources repo
!unzip 'AIMIRR-Resources/Data.zip'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp310-cp310-linux_x86_64.whl size=44090 sha256=106b1434caf7a295bdf9c5b9f06840f59e24721a90f6690b427d5e638ac577fd
  Stored in directory: /root/.cache/pip/wheels/70/83/31/975b737609aba39a4099d471d5684141c1fdc3404f97e7f68a
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.12.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
flax 0.6.10 requires PyYAML>=5.4.1, but you have pyyaml 5.1 which is incompatib

In [2]:
#importing most important libraries and processes and checking their versions

import torch, detectron2, torchvision, pandas
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
print("torchvision", torchvision.__version__)
print("pandas", pandas.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6
torchvision 0.15.2+cu118
pandas 1.5.3


In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
import math
import pandas as pd
from torchvision import transforms as T

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [4]:
#loading specific keypoint mask rcnn model
model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=True)

# eval() method to prepares model for inference mode
model.eval()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=KeypointRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=KeypointRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/keypointrcnn_resnet50_fpn_coco-fc266e95.pth" to /root/.cache/torch/hub/checkpoints/keypointrcnn_resnet50_fpn_coco-fc266e95.pth
100%|██████████| 226M/226M [00:02<00:00, 88.8MB/s]


KeypointRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(640, 672, 704, 736, 768, 800), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.

In [5]:
#keypoint function that narrows detected persons to confidence score above 90 and returning pixel arrays as integer arrays from Tensor Arrays
def keypoint_extraction(keypoints, confs):
  for person_id in range(len(keypoints)):
      if confs[person_id]>0.9:
        keypoints = keypoints[person_id].detach().numpy().tolist()
        keypoints = [np.delete(body_point, 2, 0) for body_point in keypoints]
        return keypoints

def arm_length(keypoints, eye_ratio):
  shoulder_to_elbow_avg = (math.dist(keypoints[6], keypoints[8]) + math.dist(keypoints[6], keypoints[5]))/2
  elbow_to_wrist_avg = (math.dist(keypoints[8], keypoints[10]) + math.dist(keypoints[7], keypoints[9]))/2
  return (shoulder_to_elbow_avg + elbow_to_wrist_avg) * eye_ratio * 0.393701

def hip_length(keypoints, eye_ratio):
  right_to_left = math.dist(keypoints[11], keypoints[12])
  return right_to_left * eye_ratio * 2 * 0.393701

def chest_circumference(keypoints, eye_ratio):
  right_to_left = math.dist(keypoints[5], keypoints[6])
  return right_to_left * eye_ratio * 3.5 * 0.393701

def waist_circumference(keypoints, eye_ratio):
  return hip_length(keypoints, eye_ratio) * 2.5 * 0.393701


In [6]:
def inference_on_dataset(path, participant_ID):
  transform = T.Compose([T.ToTensor()])
  img_tensor = transform(cv2.imread(path))
  output = model([img_tensor])[0]
  keypoints = keypoint_extraction(output["keypoints"], output["scores"])

  actual_eye_to_eye = keypoints[1][0] - keypoints[2][0]
  average_eye_to_eye = 6.2
  eye_ratio = average_eye_to_eye/actual_eye_to_eye

  return [participant_ID,
          chest_circumference(keypoints, eye_ratio),
          waist_circumference(keypoints, eye_ratio),
          arm_length(keypoints, eye_ratio),
          hip_length(keypoints, eye_ratio)]

In [8]:
df_data = pd.read_excel('AIMIRR-Resources/Measurements.xlsx')
column_data = df_data['Participant ID']

data = {'participant ID' : [],
        'chest circumference': [],
        'waist circumference' : [],
        'arm length': [],
        'hip length' : []}

df = pd.DataFrame(data)

for participant_ID in column_data:
  path = "./Data/" + str(participant_ID) + "/front.png"

  print(path)

  new_row = pd.Series(inference_on_dataset(path, participant_ID), index=['participant ID', 'chest circumference', 'waist circumference', 'arm length', 'hip length'])
  df = pd.concat([df, new_row.to_frame().T], ignore_index=True)

df.to_csv('output.csv', index=False)

print("\n after code block is finished you can download the output.csv file by accessing the file directories and downloading it from there")

./Data/1001/front.png
./Data/1002/front.png
./Data/1003/front.png
./Data/1004/front.png
./Data/1005/front.png
./Data/1006/front.png
./Data/1007/front.png
./Data/1008/front.png
./Data/1009/front.png
./Data/1010/front.png
./Data/1011/front.png
./Data/1012/front.png
./Data/1013/front.png
./Data/1014/front.png
./Data/1015/front.png
./Data/1016/front.png
./Data/1017/front.png
./Data/1018/front.png
./Data/1019/front.png
./Data/1020/front.png
./Data/1021/front.png
./Data/1022/front.png
./Data/1023/front.png

 after code block is finished you can download the output.csv file by accessing the file directories and downloading it from there
